# Introdução

Nesse momento era necessário decidir qual caminho eu deveria seguir para uma classificação com Machine Learning.
  
Com base na redução brusca do N à medida que são inseridos dados sobre fatores contribuintes e recomendações, além da complexidade de estruturar um DataSet para predição de completude e/ou estruturação de recomendações em meio ao pouco tempo para o desafio, decidi deixar classificações sobre recomendações e relatórios no campo das ideias.
  
  
**Com isso, seguirei para uma classificação de 3 classes, onde a partir de informações da ocorrência e da aeronave irei predizer qual a classificação do acontecido.**

**Motivação**: Automatizar ou criar um sistema de sugestão para a classificação da ocorrência

## Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np

----------------

# Funções de suporte

Função para transformar valores de String em int no estilo "sparse". Aqui será útil em um caso.

In [2]:
def make_int(col):
    if col.dtype == 'object': # Verifica se a coluna é de string (ou DateTime)
        col = col.replace(np.nan,'null') # Transforma nans em string para serem mapeados
        c_map = col.map({k: i for i, k in enumerate(np.unique(col))}) # Mapeia os valores
        c_map['null'] = np.nan # Destransforma os nulls
        return c_map
    return col

----------------

# Carregando DataSet

Para esse problema iremos utilizar o "Cenipa1", que é o DataSet que considera apenas ocorrência, ocorrência_tipo e aeronave.

In [3]:
cenipa1 = pd.read_csv('../DataSets/merged/CENIPA1.csv')

# Selecionando features

In [4]:
df = cenipa1[['ocorrencia_classificacao', # Target class
             'ocorrencia_aerodromo', # Aerodromos pequenos podem transformar um incidente em um incidente grave
             'ocorrencia_hora', # A hora da ocorrencia pode aumentar o perigo de machucar pessoas em terra-firme
             'total_aeronaves_envolvidas',
             'ocorrencia_saida_pista',
             'ocorrencia_tipo',
             'ocorrencia_tipo_categoria',
             'taxonomia_tipo_icao', # Tipo do aerodromo, se entendi bem
             'aeronave_tipo_veiculo', # Um helicoptero caindo provavelmente vai matar menos gente que um aviao caindo
             'aeronave_fabricante', # Fabricante talvez seja "forçar a barra", mas talvez faça sentido
             'aeronave_modelo',
             'aeronave_tipo_icao', # Tipo da aeronave, se entendi bem
             'aeronave_motor_tipo',
             'aeronave_motor_quantidade',
             'aeronave_pmd',
             'aeronave_pmd_categoria',
             'aeronave_assentos',
             'aeronave_ano_fabricacao',
             'aeronave_registro_segmento',
             'aeronave_voo_origem',
             'aeronave_voo_destino',
             'aeronave_fase_operacao',
             'aeronave_tipo_operacao',
             'aeronave_nivel_dano',
             'aeronave_fatalidades_total']].dropna(axis=1, thresh=len(cenipa1)*0.9) 
                                           # Se alguma dessas colunas tem mais de 90% de dados faltantes, ela não entra

# Visualizando Head e Info

In [5]:
df.head(10)

,ocorrencia_classificacao,ocorrencia_hora,total_aeronaves_envolvidas,ocorrencia_saida_pista,ocorrencia_tipo,ocorrencia_tipo_categoria,taxonomia_tipo_icao,aeronave_tipo_veiculo,aeronave_fabricante,aeronave_modelo,...,aeronave_pmd_categoria,aeronave_assentos,aeronave_ano_fabricacao,aeronave_registro_segmento,aeronave_voo_origem,aeronave_voo_destino,aeronave_fase_operacao,aeronave_tipo_operacao,aeronave_nivel_dano,aeronave_fatalidades_total
0,INCIDENTE,20.0,1,NÃO,ESTOURO DE PNEU,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP,AVIÃO,RAYTHEON AIRCRAFT,58,...,2495,6.0,2003.0,PARTICULAR,FORA DE AERODROMO,FORA DE AERODROMO,POUSO,PRIVADA,LEVE,0
1,ACIDENTE,13.0,1,NÃO,COM PESSOAL EM VOO,OUTROS | COM PESSOAL EM VOO,OTHR,AVIÃO,AEROSPATIALE AND ALENIA,ATR-42-500,...,18600,50.0,2001.0,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,DESCIDA,REGULAR,NENHUM,0
2,ACIDENTE,13.0,1,NÃO,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,EMB-201,...,1800,1.0,1976.0,ESPECIALIZADA,FORA DE AERODROMO,FORA DE AERODROMO,ESPECIALIZADA,AGRÍCOLA,SUBSTANCIAL,0
3,ACIDENTE,17.0,1,NÃO,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP,ULTRALEVE,NaN,P2004 BRAVO,...,580,2.0,2007.0,EXPERIMENTAL,FORA DE AERODROMO,FORA DE AERODROMO,CRUZEIRO,EXPERIMENTAL,LEVE,0
4,ACIDENTE,16.0,1,NÃO,PERDA DE CONTROLE NO SOLO,PERDA DE CONTROLE NO SOLO,LOC-G,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,EMB-201A,...,1800,1.0,1986.0,ESPECIALIZADA,FORA DE AERODROMO,FORA DE AERODROMO,POUSO,AGRÍCOLA,SUBSTANCIAL,0
5,INCIDENTE,14.0,1,NÃO,COLISÃO COM AVE,COLISÃO COM AVE,BIRD,HELICÓPTERO,EUROCOPTER FRANCE,EC 120 B,...,1715,5.0,2000.0,PARTICULAR,FORA DE AERODROMO,FORA DE AERODROMO,CRUZEIRO,PRIVADA,LEVE,0
6,INCIDENTE GRAVE,18.0,1,NÃO,PERDA DE CONTROLE NO SOLO,PERDA DE CONTROLE NO SOLO,LOC-G,AVIÃO,CIA AERONAUTICA PAULISTA,CAP-4,...,587,2.0,1940.0,INSTRUÇÃO,FORA DE AERODROMO,FORA DE AERODROMO,DECOLAGEM,INSTRUÇÃO,SUBSTANCIAL,0
7,INCIDENTE,19.0,1,NÃO,ESTOURO DE PNEU,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP,AVIÃO,AIRBUS INDUSTRIE,A320-214,...,77000,184.0,2008.0,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,CORRIDA APÓS POUSO,REGULAR,LEVE,0
8,ACIDENTE,16.0,1,NÃO,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP,AVIÃO,CESSNA AIRCRAFT,P210N,...,1812,6.0,1982.0,PARTICULAR,FORA DE AERODROMO,FORA DE AERODROMO,CRUZEIRO,PRIVADA,SUBSTANCIAL,0
9,INCIDENTE,22.0,1,NÃO,COM TREM DE POUSO,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP,AVIÃO,JOSE ROBERTO BARBOSA,BUMERANGUE EX-27 CROSS-CONTRY,...,1260,4.0,2011.0,EXPERIMENTAL,FORA DE AERODROMO,FORA DE AERODROMO,POUSO,EXPERIMENTAL,LEVE,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5417 entries, 0 to 5416
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ocorrencia_classificacao    5417 non-null   object 
 1   ocorrencia_hora             5416 non-null   float64
 2   total_aeronaves_envolvidas  5417 non-null   int64  
 3   ocorrencia_saida_pista      5417 non-null   object 
 4   ocorrencia_tipo             5416 non-null   object 
 5   ocorrencia_tipo_categoria   5414 non-null   object 
 6   taxonomia_tipo_icao         5416 non-null   object 
 7   aeronave_tipo_veiculo       5258 non-null   object 
 8   aeronave_fabricante         5059 non-null   object 
 9   aeronave_modelo             5242 non-null   object 
 10  aeronave_tipo_icao          5146 non-null   object 
 11  aeronave_motor_tipo         5180 non-null   object 
 12  aeronave_motor_quantidade   5323 non-null   object 
 13  aeronave_pmd                5417 

# Preenchendo Dados Faltantes

**AVISO**: Para menter a integridade total da validação, tipicamente separamos o treino da validação/teste **antes** de fazer o *preenchimento* de dados. Dessa forma, podemos criar uma situação mais real e apenas preencher os dados faltantes a partir dos dados de treino, mantendo a "imprevisibilidade" dos dados de teste. No entanto, para esse DataSet específico, pela alta quantidade de dados categóricos interconectados que não afetam exclusivamente a target class (como modelo e quantidade de assentos da aeronave, por exemplo), não há grandes problemas em preencher tudo antes dessa divisão.

**Algumas colunas nunca estão nulas**:
- *ocorrencia_classificacao*
- *total_aeronaves_envolvidas*
- *ocorrencia_saida_pista*
- *aeronave_pmd*
- *aeronave_pmd_categoria*
- *aeronave_fatalidades_total*

**Outras colunas não dispoem de informação o suficiente para serem preenchidas satisfatoriamente**:
- *ocorrencia_tipo*
- *ocorrencia_tipo_categoria*
- *taxonomia_tipo_icao*
- *aeronave_fase_operacao*
- *aeronave_tipo_operacao*
- *aeronave_nivel_dano*

Vamos manter linhas com esses valores vazios pois iremos criar o dataset com pd.getdummies()

In [7]:
def fill_missing(og_df, corr):
    
    filled_df = og_df.copy() # Iremos retornar o dataframe preenchido
    
    # Preenchendo com valores constantes
    # Não é possível prever a hora a partir de outros dados consistentemente, por isso inserimos a moda
    filled_df['ocorrencia_hora'].fillna(og_df['ocorrencia_hora'].mode()[0], inplace=True)
    
    # Origem e destino do voo são faltantes quando não se consegue pontuar claramente (premissa)
    filled_df['aeronave_voo_origem'].fillna('FORA DE AERODRAMO', inplace=True)
    filled_df['aeronave_voo_destino'].fillna('FORA DE AERODRAMO', inplace=True)
    
    fillable_cols = ['aeronave_tipo_veiculo', 'aeronave_fabricante', 'aeronave_modelo', 'aeronave_tipo_icao',
                    'aeronave_motor_tipo', 'aeronave_motor_quantidade', 'aeronave_assentos', 'aeronave_ano_fabricacao',
                    'aeronave_registro_segmento']
    
    # Para cada valor no dataframe
    for i,row in og_df.iterrows():
        
        # E para cada variável
        for col in fillable_cols:
            
            # Se não for NaN, skip
            if not row[col]!=row[col]:
                continue
            
            # Nome do modelo faltando
            if col=='aeronave_modelo':
                
                # Cria um df com as aeronaves de mesmo fabricante ou mesmo icao
                sup_df = og_df[(og_df['aeronave_fabricante']==row['aeronave_fabricante'])|
                           (og_df['aeronave_tipo_icao']==row['aeronave_tipo_icao'])]
                
                try:
                    filled_df.loc[i,col] = sup_df[col].mode()[0] # E preenche com a moda do sup_df
                except:
                    filled_df.loc[i,col] = np.nan # Se não der, continua NaN
              
            # Demais casos
            else:
                
                # Cria um df igualando uma das 3 colunas de maior correlacao com a variavel
                col_corr = pd.DataFrame(corr[col].abs().sort_values(ascending=False)[1:6])
                highest_corrs = col_corr.index.tolist()
                
                sup_df = og_df[(og_df[highest_corrs[0]]==row[highest_corrs[0]])|
                              (og_df[highest_corrs[1]]==row[highest_corrs[1]])|
                              (og_df[highest_corrs[2]]==row[highest_corrs[2]])|
                              (og_df[highest_corrs[1]]==row[highest_corrs[3]])|
                              (og_df[highest_corrs[1]]==row[highest_corrs[4]])]
                
                try:
                    filled_df.loc[i,col] = sup_df[col].mode()[0] # E preenche com a moda do sup_df
                except:
                    filled_df.loc[i,col] = np.nan # Se não der, mantém o NaN
    
    # Os únicos valores que não podem ser NaN nunca são colunas numéricas
    return filled_df.dropna(how='any',subset=['aeronave_assentos', 'aeronave_ano_fabricacao']).reset_index(drop=True)

In [8]:
df=fill_missing(df.copy(),df.apply(make_int).corr())

In [10]:
df.to_csv('../DataSets/ML/cenipa1.csv',index_label=False)